In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline

In [4]:
accre_jobs_2020 = pd.read_csv('../data/accre_jobs_2020_cleaned.csv')
display(accre_jobs_2020.head())

,jobid,account,user,reqtime,usedtime,nodes,cpus,partition,state,exitcode_user,exitcode_error,used_mb_per_core,reqmem_per_core,cluster_type,cluster_nodes
0,15925210,treviso,arabella,1188000,1188028,1,24,production,COMPLETED,0,0,2748.895417,5120.0,cn,1531
1,15861126,treviso,arabella,1188000,1090256,1,24,production,COMPLETED,0,0,2799.213333,5120.0,cn,1441
2,15861125,treviso,arabella,1188000,1188020,1,24,production,COMPLETED,0,0,2879.660833,5120.0,cn,1464
3,16251645,treviso,arabella,1188000,1050632,1,24,production,COMPLETED,0,0,2721.555417,5120.0,cn,1473
4,16251646,treviso,arabella,1188000,1188003,1,24,production,COMPLETED,0,0,2744.837917,5120.0,cn,1440


# Exploring the dataframe

In [5]:
accre_jobs_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3813160 entries, 0 to 3813159
Data columns (total 15 columns):
jobid               object
account             object
user                object
reqtime             int64
usedtime            int64
nodes               int64
cpus                int64
partition           object
state               object
exitcode_user       int64
exitcode_error      int64
used_mb_per_core    float64
reqmem_per_core     float64
cluster_type        object
cluster_nodes       object
dtypes: float64(2), int64(6), object(7)
memory usage: 436.4+ MB


## column state

In [6]:
accre_jobs_2020['state'].value_counts()

COMPLETED              3804625
CANCELLED                 6106
FAILED                    1404
CANCELLED by 686562        681
OUT_OF_MEMORY               94
TIMEOUT                     80
CANCELLED by 505355         63
CANCELLED by 855431         17
CANCELLED by 782611         13
CANCELLED by 9206            9
CANCELLED by 397600          7
CANCELLED by 200557          6
CANCELLED by 0               5
CANCELLED by 486541          4
CANCELLED by 454080          4
CANCELLED by 199766          4
CANCELLED by 9201            3
CANCELLED by 9202            3
CANCELLED by 483348          3
CANCELLED by 649319          3
CANCELLED by 693461          2
CANCELLED by 90423           2
CANCELLED by 791651          2
CANCELLED by 666860          2
CANCELLED by 895426          2
CANCELLED by 546080          2
CANCELLED by 124006          2
CANCELLED by 199066          2
CANCELLED by 515423          2
CANCELLED by 651701          1
CANCELLED by 763605          1
CANCELLED by 337422          1
CANCELLE

In [7]:
### trying to add cancelled by ... to canceled jobs
#re.sub(r"CANCELLED by\s\d+", 'CANCELLED', accre_jobs_2020.state) need to do a lambda function and .apply
#accre_jobs_2020[accre_jobs_2020["account"].str.match('\d+')].value_counts()

In [8]:
#accre_jobs_2020[(accre_jobs_2020_clean2['state'] == 'FAILED')].nodelist.value_counts()

In [9]:
#plt.figure(figsize = (20,500))
#accre_jobs_2020[(accre_jobs_2020['state'] == 'FAILED')].groupby('state')['nodelist'].value_counts().plot(kind = 'barh')
#plt.yticks(fontsize = 20);

## column partition

In [10]:
accre_jobs_2020['partition'].value_counts()

production              3308915
nogpfs                   327432
sam                       79151
pascal                    48000
turing                    39377
debug                      6738
maxwell                    3347
cgw-capra1                   83
cgw-dougherty1               70
cgw-cqs1                     34
cgw-cqs3                      4
cgw-vm-qa-flatearth1          4
devel                         2
cgw-tbi01                     2
cgw-rocksteady                1
Name: partition, dtype: int64

## Exit code error

In [ ]:
#accre_jobs_2020(accre_jobs_2020['exitcode_error'] != 0)

In [24]:
accre_jobs_2020['account'].value_counts()

cms              806448
cep              609282
summer           380840
galia            198036
carrot           192551
orange           158400
cms_lowprio      135682
casaba           117703
portabella       109680
bunch            105877
cmsadmin         102589
bulk              78608
boysenberries     57620
sharlin           57397
horseradish       53707
grisette          42447
sweet             40241
mixed             39311
retailer          39091
key               37711
round             36827
wood              32383
savoy             30482
sprite            25892
atemoyas          25020
winged            24770
garlic            19969
wax               19623
celery            17399
alkmene           15605
                  ...  
pickling             11
maltaise             11
galangal             10
oroblanco             9
pioppino              7
polk                  7
nino                  6
cocktail              6
ingrid                6
stalk                 4
pink            

In [27]:
accre_jobs_2020.groupby('account')['user'].value_counts()

account   user     
alkmene   velvet       15269
          vaughn         163
          marci          138
          abigail         23
          cleve           12
almonds   montie        1891
amaranth  alvira        3033
angelys   namon          899
          macey            9
          katrina          4
          christian        3
          auther           2
          lethia           1
anise     ginger        1180
          kori           225
          osbaldo        182
          adelina        116
          sigrid          48
          alvera          17
          hattie           3
          hailey           1
antares   hubbard        160
arra      needham         65
atemoyas  pamelia      25018
          loda             2
baby      kegan         3186
baking    suzie          137
          christa        116
          gerhardt        98
          gladstone       74
                       ...  
water     mareli          46
wax       porfirio     19623
white     pearla       

# Questions:

### Question 1: 
While we have systematic checks in place to ensure the general system health of each compute node, we would like to use long-term data to see if there are any clusters of job failures on specific nodes. Do any of the production partition nodes show an unusual number of failed jobs relative to the others? Ignore the debug partition for this question.
We are concerned with potentially unreliable nodes which are not being detected by our routine monitoring. Of the failed jobs, do any nodes show up unusually often? 

•	Look for failed jobs

In [11]:
accre_jobs_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3813160 entries, 0 to 3813159
Data columns (total 15 columns):
jobid               object
account             object
user                object
reqtime             int64
usedtime            int64
nodes               int64
cpus                int64
partition           object
state               object
exitcode_user       int64
exitcode_error      int64
used_mb_per_core    float64
reqmem_per_core     float64
cluster_type        object
cluster_nodes       object
dtypes: float64(2), int64(6), object(7)
memory usage: 436.4+ MB


In [13]:
accre_jobs_2020_failed = accre_jobs_2020.loc[(accre_jobs_2020['state'] == 'FAILED')]

In [14]:
accre_jobs_2020_failed.shape

(1404, 15)

In [16]:
accre_jobs_2020_failed['partition'].value_counts()

pascal        702
production    500
maxwell       102
nogpfs         53
debug          27
turing         20
Name: partition, dtype: int64

In [17]:
accre_jobs_2020_failed['cluster_type'].value_counts()

gpu        825
cn         573
ng           4
generic      2
Name: cluster_type, dtype: int64

In [18]:
accre_jobs_2020_failed[(accre_jobs_2020_failed["partition"] == "production")].cluster_type.value_counts()

cn    500
Name: cluster_type, dtype: int64

In [21]:
accre_jobs_2020_failed_prod.cluster_nodes.__len__()

500

In [ ]:
#accre_jobs_2020[(accre_jobs_2020['state'] == 'FAILED') & (accre_jobs_2020["partition"] == "turing")].nodelist.value_counts()

In [ ]:
#accre_jobs_2020_failed[(accre_jobs_2020_failed["partition"] == "production") & (accre_jobs_2020_failed["cluster_type"] == "cn")].cluster_nodes.value_counts()

In [19]:
accre_jobs_2020_failed_prod = accre_jobs_2020_failed.loc[accre_jobs_2020_failed["partition"] == "production"]

In [ ]:
#accre_jobs_2020_failed2 = accre_jobs_2020_failed.groupby['cluster_type']('cluster_nodes')

In [20]:
#pd.set_option('display.max_rows', None)
#accre_jobs_2020_failed_prod.cluster_nodes.value_counts()
accre_jobs_2020_failed_prod['cluster_nodes'].value_counts().to_frame()

,cluster_nodes
1273,9
1458,6
1364,6
304,6
1450,5
1270,5
1479,5
"1530,1531",5
"1468,1469",4
1267,4


In [37]:
def node_list_count (cluster_nodes, x):
    """This function is to count the number of occurence of single nodes in cluster nodes."""
    count = 0
    if (element == x):
        count = count + 1            
    return count 

In [43]:
def countX(cluster_nodes, x): 
    count = 0
    for ele in lst: 
        if (ele == x): 
            count = count + 1
    return count

In [45]:
countX('cluster_nodes', 1273)

NameError: name 'lst' is not defined

In [41]:
node_list_count(cluster_nodes, '1273')

NameError: name 'cluster_nodes' is not defined

In [ ]:
def node_list_count (cluster_nodes, x):
    """This function is to count the number of occurence of single nodes in cluster nodes."""
    count = 0
    if len(cluster_nodes) = 1:
      for element in cluster_nodes:  
        if (element == x):
            count = count + 1            
    return count 

"""To get a list of all the rows of all nodes and cluster nodes. Nb= cluster nodes are in [ ]"""
### Create an empty list 
Row_list =[] 
  
### Iterate over each row 
for index, rows in accre_jobs_2020_failed_prod.iterrows(): 
    # Create list for the current row 
    cluster_nodes_list =[rows.cluster_nodes] 
      
    # append the list to the final list 
    Row_list.append(cluster_nodes_list) 
  
### Print the list 
print(Row_list)

"""To remove [ ] in the list"""
flatten_matrix = [ ] 
  
for sublist in Row_list: 
    for val in sublist: 
        flatten_matrix.append(val) 
          
print(flatten_matrix) 

In [22]:
###to remove ' ' between all numbers
#no_commas = ",".join(flatten_matrix)

In [ ]:
### to find all numbers that are the same
#find_numbers = re.findall(r"\d+", no_commas)

In [ ]:
### to count the list of numbers
#from collections import Counter
#count_nodes = Counter(find_numbers)

In [ ]:
### I need it to make a series
#from pandas import Series
#series_nodes = Series(count_nodes)
#series_nodes

In [ ]:
#type(series_nodes)

In [ ]:
#series_nodes.to_frame()

In [ ]:
#nodes_index = d.reset_index()

In [ ]:
#tried to rename the columns but it didn't work because 
#final_nodes = nodes_index.rename(columns = {'nodes_number':'fail_Counts'})

In [ ]:
#final_nodes.plot(kind='bar');